In [15]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms

Building a network with -
2 Hidden layers and one output layer 

In [49]:
class Net(nn.Module):
    
    #initialising the network to have a random linear functions. Generating a whole NN.
    def __init__(self,D_in,H1,H2,D_out):
        super(Net,self).__init__()
        #now i have created a child class of Module. 
        #I can call functions of parent class 
        #and store it as objects of child class
        self.Linear1=nn.Linear(D_in,H1)
        self.Linear2=nn.Linear(H1,H2)
        #the represents the output layer
        self.Linear3=nn.Linear(H2,D_in)
    def forward(self,x):
        x=torch.sigmoid(self.Linear1(x))
        x=torch.sigmoid(self.Linear2(x))
        x=self.Linear3(x)
        return x

**Remember**  
- If you want to implement examples in one node. You can put every example in each row of a tensor torch
- And if you want to put different features. Its always better to put them in different nodes all together

In [17]:
#this has popular training datasets
train_dataset=dset.MNIST(root='./data',train=True,download=True,transform=transforms.ToTensor())

0it [00:00, ?it/s]

9920512it [00:28, 336476.89it/s]                             

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz



0it [00:00, ?it/s]


 57%|█████▋    | 16384/28881 [00:00<00:00, 58095.00it/s]

0it [00:00, ?it/s]

...



  0%|          | 0/1648877 [00:00<?, ?it/s]

  1%|          | 16384/1648877 [00:00<00:29, 55447.64it/s]

  2%|▏         | 40960/1648877 [00:01<00:25, 62427.80it/s]

  6%|▌         | 98304/1648877 [00:01<00:19, 78453.60it/s]

 11%|█         | 180224/1648877 [00:01<00:14, 99594.27it/s]

 15%|█▌        | 253952/1648877 [00:02<00:11, 123962.62it/s]

 18%|█▊        | 294912/1648877 [00:02<00:08, 154608.42it/s]

 21%|██        | 344064/1648877 [00:02<00:07, 171063.23it/s]

 25%|██▍       | 409600/1648877 [00:02<00:05, 216354.56it/s]

 27%|██▋       | 450560/1648877 [00:02<00:05, 202872.99it/s]

 32%|███▏      | 524288/1648877 [00:02<00:04, 227888.56it/s]

 34%|███▍      | 565248/1648877 [00:03<00:04, 256015.98it/s]

 36%|███▋      | 598016/1648877 [00:03<00:05, 176530.80it/s]

 43%|████▎     | 704512/1648877 [00:03<00:04, 216725.48it/s]

 45%|████▌     | 745472/1648877 [00:03<00:03, 243134.05it/s]

 47%|████▋     | 778240/1648877 [00:03<00:03, 229088.98it/s]

 50%|████▉     | 819200/164887

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz




  0%|          | 0/4542 [00:00<?, ?it/s]

8192it [00:00, 13451.88it/s]            

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


9920512it [00:40, 336476.89it/s]
32768it [00:20, 58095.00it/s]                           

In [18]:
## setting a loss function
criterion=nn.CrossEntropyLoss()

In [19]:
# Create the model class using Relu as a activation function

class NetRelu(nn.Module):
    
    # Constructor
    def __init__(self, D_in, H1, H2, D_out):
        super(NetRelu, self).__init__()
        self.linear1 = nn.Linear(D_in, H1)
        self.linear2 = nn.Linear(H1, H2)
        self.linear3 = nn.Linear(H2, D_out)
    
    # Prediction
    def forward(self, x):
        x = F.relu(self.linear1(x))  
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

In [20]:
# Train the model
# Trainloader lets you extract the data in x and y

def train(model, criterion, train_loader, validation_loader, optimizer, epochs=100):
    i = 0
    useful_stuff = {'training_loss': [], 'validation_accuracy': []}  
    
    for epoch in range(epochs):
        for i, (x, y) in enumerate(train_loader):
            optimizer.zero_grad()
            z = model(x.view(-1, 28 * 28))
            loss = criterion(z, y)
            loss.backward()
            optimizer.step()
            useful_stuff['training_loss'].append(loss.data.item())
        
        correct = 0
        for x, y in validation_loader:
            yhat = model(x.view(-1, 28 * 28))
            _, label = torch.max(yhat, 1)
            correct += (label == y).sum().item()
    
        accuracy = 100 * (correct / len(validation_dataset))
        useful_stuff['validation_accuracy'].append(accuracy)
    
    return useful_stuff

In [28]:
# Create the validating dataset

validation_dataset = dset.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())


In [30]:
# Set the parameters for create the model
# image is of dimension 28*28. It feeds in the network. Output is of a digit from 1 to 10.

input_dim = 28 * 28
hidden_dim1 = 50
hidden_dim2 = 50
output_dim = 10

In [31]:
# Set the number of iterations

cust_epochs = 10

In [32]:
# Create the training data loader and validation data loader object

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=2000, shuffle=True)
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, batch_size=5000, shuffle=False)

In [33]:
train_loader

[Jist of optimisers](https://pytorch.org/docs/stable/optim.html)

In [52]:
# Train the model with sigmoid function

learning_rate = 0.01
model = Net(input_dim, hidden_dim1, hidden_dim2, output_dim)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
training_results = train(model, criterion, train_loader, validation_loader, optimizer, epochs=cust_epochs)

In [51]:
training_results

{'training_loss': [6.711361408233643,
  6.719888210296631,
  6.698922634124756,
  6.669858932495117,
  6.659979343414307,
  6.645028591156006,
  6.636767387390137,
  6.615419387817383,
  6.600926399230957,
  6.591947555541992,
  6.576782703399658,
  6.5601911544799805,
  6.550868988037109,
  6.541611194610596,
  6.51452112197876,
  6.503509998321533,
  6.4915852546691895,
  6.477435111999512,
  6.451359748840332,
  6.446980953216553,
  6.435943126678467,
  6.4193315505981445,
  6.408897876739502,
  6.396498680114746,
  6.3775835037231445,
  6.365340232849121,
  6.365054607391357,
  6.3460235595703125,
  6.342011451721191,
  6.307867050170898,
  6.309206485748291,
  6.286592960357666,
  6.266880035400391,
  6.256866931915283,
  6.245728969573975,
  6.225630283355713,
  6.220235824584961,
  6.200465202331543,
  6.191842079162598,
  6.167851448059082,
  6.161558151245117,
  6.13621187210083,
  6.129256248474121,
  6.121739387512207,
  6.103392124176025,
  6.102561950683594,
  6.0932879447